In [2]:
import pandas as pd
import numpy as np 
import aux_functions
from multiprocess import Pool 
import ast
from bpemb import BPEmb
import country_converter
from rapidfuzz import fuzz
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, precision_recall_fscore_support, precision_recall_curve, accuracy_score
from sklearn.linear_model import LogisticRegression
from plotnine import *
from sklearn.metrics import SCORERS
from sklearn.ensemble import RandomForestClassifier

In [3]:
import importlib
importlib.reload(aux_functions)

<module 'aux_functions' from '/workspace/src/automatic_matching/aux_functions.py'>

In [4]:
# Download model
bpemb_multi = BPEmb(lang="multi", vs=320000, dim=300)

In [9]:
bpemb_multi.encode('Im eating and running in Stratford, juggling')

['▁im',
 '▁eating',
 '▁and',
 '▁running',
 '▁in',
 '▁stratford',
 ',',
 '▁jug',
 'g',
 'ling']

## Read data

In [20]:
matching_data = pd.read_csv('matching_results.tsv', sep='\t')
provider_responses_df = pd.read_csv('provider_responses_df.csv')

cols_drop = ['upload_date', 'apiversion', 'provider_api_id', 'provider_lat',
             'provider_lon', 'lat', 'lon', 'populated_fields', 'osm_class',
             'osm_subtype', 'osm_id', 'api1_run_id']
matching_data = matching_data.drop(columns=cols_drop).rename(columns={'country':'country_name'})

## Apply Libpostal to searched query

In [21]:
# Create searched_query dataframe
searched_query_df = matching_data[['searched_query']].drop_duplicates()


searched_query_df['libpostal_query'] = (searched_query_df
                                        .searched_query
                                        .apply(aux_functions.column_formatted_address_to_query_dict)
                                        )

with Pool(10) as p:
    libpostal_response = list(p.imap(aux_functions.column_query_to_response,
                                     searched_query_df.libpostal_query.tolist())
                             )
    
# Get libpostal components for searched query
searched_query_df['libpostal_response'] = libpostal_response
libpostal_components = searched_query_df.libpostal_response.apply(lambda x: pd.Series(ast.literal_eval(x)))
searched_query_df[libpostal_components.columns] = libpostal_components.fillna('')


matching_data = matching_data.merge(searched_query_df)

In [22]:
matching_data_features = aux_functions.get_provider_components(matching_data)

## Feature engineering

For simplicity, we will rule out the addresses where the distance was bigger than 5 kms. This will retain matches of higher qualities and also reduce the search space

In [15]:
matching_final_df = matching_data_features.loc[matching_data_features.provider_distance <= 5000].reset_index(drop=True)
matching_final_df = matching_final_df.merge(provider_responses_df.fillna('').astype(str).drop(columns=['libpostal_query', 'libpostal_response']),
                                            how='left',
                                            on='provider_formatted_address')
matching_final_df = matching_final_df.loc[matching_final_df.house_number!=''].reset_index(drop=True)

# Ensure components are string
matching_final_df[['postcode', 'provider_pc_component', 'provider_postcode']] = (matching_final_df[['postcode', 'provider_pc_component', 'provider_postcode']]
                                                                                 .astype(str))
matching_final_df[['road', 'provider_road_component', 'provider_road']] = (matching_final_df[['road', 'provider_road_component', 'provider_road']]
                                                                           .astype(str))
matching_final_df[['house_number', 'provider_house_number', 'house_number_component']] = (matching_final_df[['house_number', 'provider_house_number', 'house_number_component']]
                                                                                          .astype(str))

# Generate features for components similarities
matching_final_df = aux_functions.build_features_similarities(matching_final_df)

## Train test split

An initial train-test split will be performed at an address level. This is, no address found on the train set will be found on the test set.

In [23]:
# Define unique addresses
unique_addresses = matching_final_df.searched_query.unique()


# Train will be 70% of the unique addresses and test the remaining one
np.random.seed(12)
train_size = 0.8
train_addresses = np.random.choice(unique_addresses, size=int(train_size*unique_addresses.shape[0]))
test_addresses = unique_addresses[~np.isin(unique_addresses,train_addresses)]


# Get dataframe
train_df = matching_final_df.loc[matching_final_df.searched_query.isin(train_addresses)].sample(frac=1, random_state=12).reset_index(drop=True)
test_df = matching_final_df.loc[matching_final_df.searched_query.isin(test_addresses)].sample(frac=1, random_state=12).reset_index(drop=True)

In [24]:
features = ['pc_similarity', 'road_similarity', 'house_number_similarity', 'locality_similarity_mean', 'provider_distance', 'global_similarity', 'return_rank']

In [25]:
# Get global similarity
train_df['global_similarity'] = list(map(fuzz.WRatio, train_df.searched_query, train_df.provider_formatted_address))
test_df['global_similarity'] = list(map(fuzz.WRatio, test_df.searched_query, test_df.provider_formatted_address))


train_features = train_df[features]; y_train = train_df.match
test_features = test_df[features]; y_test = test_df.match


# Impute PC with a mean score of other similarities
train_features.loc[:, 'pc_similarity'] = np.where(train_features.pc_similarity.isna(),
                                                  train_features[['road_similarity', 'house_number_similarity', 
                                                                  'locality_similarity_mean', 'global_similarity']].mean(axis=1),
                                                  train_features.pc_similarity)

test_features.loc[:, 'pc_similarity'] = np.where(test_features.pc_similarity.isna(),
                                                  test_features[['road_similarity', 'house_number_similarity', 
                                                                  'locality_similarity_mean', 'global_similarity']].mean(axis=1),
                                                  test_features.pc_similarity)

## Models

### Logistic Regression

In [26]:
lr = LogisticRegression(max_iter=500000)
params_lr = {'C': [10.**x for x in np.linspace(0, 1., 100)]}

In [27]:
lr_gcv = GridSearchCV(estimator=lr,
                      param_grid=params_lr,
                      scoring='f1',
                      cv=5,
                      n_jobs=-1).fit(train_features, y_train)

best_lr = lr_gcv.best_estimator_

In [33]:
lr_test_predict_proba = best_lr.predict_proba(test_features)
test_features_df = test_features.assign(provider_formatted_address = test_df.provider_formatted_address,
                                        searched_query = test_df.searched_query,
                                        provider_id = test_df.provider_id,
                                        predict_proba = lr_test_predict_proba[:,1],
                                        match=test_df.match)

In [34]:
test_features_prediction = test_features_df.loc[test_features_df.groupby(['searched_query', 'provider_id']).predict_proba.idxmax()].reset_index(drop=True)
test_features_prediction['prediction'] = np.where(test_features_prediction.predict_proba >= 0.5, 1, 0)

In [35]:
print('ROC AUC score: ', roc_auc_score(test_features_prediction.match, test_features_prediction.predict_proba))
print('F1 score: ', f1_score(test_features_prediction.match, test_features_prediction.prediction))
print('Recall: ', recall_score(test_features_prediction.match, test_features_prediction.prediction))
print('Precision: ', precision_score(test_features_prediction.match, test_features_prediction.prediction))
print('Accuracy: ', accuracy_score(test_features_prediction.match, test_features_prediction.prediction))

ROC AUC score:  0.9337567398543009
F1 score:  0.9643369542939126
Recall:  0.9781704781704782
Precision:  0.9508892481810832
Accuracy:  0.938635161347205


In [ ]:
# DCG
# Discounted cumulative gain



In [222]:
(test_features_prediction.loc[test_features_prediction.match != test_features_prediction.prediction,
                            ['searched_query', 'provider_formatted_address', 'match', 'prediction', 'predict_proba', 'provider_distance']]
 .sort_values(by='provider_distance')
 .to_csv('recalibration_matches.csv', index=False)
)

In [39]:
(test_features_prediction.loc[test_features_prediction.match != test_features_prediction.prediction,
                            ['searched_query', 'provider_formatted_address', 'match', 'prediction', 'predict_proba', 'provider_distance']]
 .sort_values(by='provider_distance')

)

searched_query  \
1350            201 Magnolia Ave, Sanford, Florida, USA   
1840                    279 WOOD ST,WILSON NY 14172-USA   
4462              ConcepciÃ³n, 14 ,02002 albacete Spain   
781            1408 SWEET HOME RD,AMHERST NY 14228, USA   
1823        2706 E Ridge Rd, Irondequoit, New York, USA   
...                                                 ...   
64    1 Rue Habib Thameur, Site archéologique de Car...   
4348  Chemin de Ravine Houel 1461, 97129 Lamentin, G...   
4247  Chemin de Desbonne 1883, Lamentin, Guadeloupe,...   
108                    10 Avenue Tidghine، Fes, Morocco   
900              1550 HIGHWAY 302  BYHALIA MS 38611,USA   

                             provider_formatted_address  match  prediction  \
1350  Ritz Theater & Wayne Densch Performing Arts Ce...      0           1   
1840  279, Wood Street, Wilson, Niagara County, New ...      0           1   
4462  Calle Concepción, 14, 02001 Albacete (Albacete...      1           0   
781   1408, Sweet Home Road, Flickinger Court, Buffa...      0           1   
1823  2706, East Ridge Road, Newport, Rochester, Mon...      0           1   
...                                                 ...    ...         ...   
64                    Rue Habib Thameur 1, Tunis, Tunis      1           0   
4348  1461 Chemin de Borel, 97129 Canton of Lamentin...      1           0   
4247  1883 Chemin de Borel, 97129 Canton of Lamentin...      1           0   
108                                10 Rue Tidghine, Fès      1           0   
900   1550 Highway 302, Byhalia, MS 38611-6308, Unit...      1           0   

      predict_proba  provider_distance  
1350       0.864579                  0  
1840       0.869569                  0  
4462       0.276656                  0  
781        0.887568                  0  
1823       0.847787                  0  
...             ...                ...  
64         0.165906               3545  
4348       0.083737               3730  
4247       0.063571               4208  
108        0.051434               4270  
900        0.095892               4459  

[348 rows x 6 columns]

### Random Forest

In [231]:
rf = RandomForestClassifier(random_state=12, n_jobs=-1)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20, 40, 80, 100, 160]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16, 32, 64, 128, 256]


# Create the random grid
params_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [235]:
rf_search = RandomizedSearchCV(rf, 
                              param_distributions=params_rf,
                              n_jobs=-1,
                              random_state=42,
                              cv=3,
                              n_iter=75, 
                              verbose=10,
                              scoring='f1').fit(train_features, y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  78 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed: 1

In [239]:
best_rf = rf_search.best_estimator_
rf_search.best_params_

{'n_estimators': 2000,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': False}

In [240]:
rf_test_predict_proba = best_rf.predict_proba(test_features)
test_features_df = test_features.assign(provider_formatted_address = test_df.provider_formatted_address,
                                        searched_query = test_df.searched_query,
                                        provider_id = test_df.provider_id,
                                        predict_proba = rf_test_predict_proba[:,1],
                                        match=test_df.match)

In [241]:
print('ROC AUC score: ', roc_auc_score(test_features_prediction.match, test_features_prediction.predict_proba))
print('F1 score: ', f1_score(test_features_prediction.match, test_features_prediction.prediction))
print('Recall: ', recall_score(test_features_prediction.match, test_features_prediction.prediction))
print('Precision: ', precision_score(test_features_prediction.match, test_features_prediction.prediction))
print('Accuracy: ', accuracy_score(test_features_prediction.match, test_features_prediction.prediction))

ROC AUC score:  0.9337567398543009
F1 score:  0.9643369542939126
Recall:  0.9781704781704782
Precision:  0.9508892481810832
Accuracy:  0.938635161347205
